# Анализ базы данных сервиса для чтения книг по подписке.

**Описание проекта** 

Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 



**Цели исследования:** 

Проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Описание данных**

**Таблица `books`**

Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.


**Задания**
1. Посчитаем, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитаем количество обзоров и среднюю оценку;
3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
4. Определим автора с самой высокой средней оценкой книг — учитём только книги с 50 и более оценками;
5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Как выполнить задание?**
- Опишем цели исследования;
- Исследуем таблицы — выведем первые строки;
- Сделаем по одному SQL-запросу для решения каждого задания;
- Выведем результаты каждого запроса в тетрадке;
- Опишем выводы по каждой из решённых задач.

#### Шаг 1. Загрузка и изучение данных

In [14]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [15]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [16]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Изучим таблицу `books`**

In [17]:
books = '''
SELECT *
FROM books
LIMIT 5;
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(books), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Изучим таблицу `authors`**

In [18]:
authors = '''
SELECT *
FROM authors
LIMIT 5;
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(authors), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Изучим таблицу `publishers`**

In [19]:
publishers = '''
SELECT *
FROM publishers
LIMIT 5;
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(publishers), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Изучим таблицу `ratings`**

In [20]:
ratings = '''
SELECT *
FROM ratings
LIMIT 5;
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(ratings), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Изучим таблицу `reviews`**

In [21]:
reviews = '''
SELECT *
FROM reviews
LIMIT 5;
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(reviews), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Вывод по разделу 1:**

Таблица `books` - содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Таблица `authors` - содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

Таблица `publishers` - содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Таблица `ratings` - содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Таблица `reviews` - содержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

#### Шаг 2. Решение задач по проекту 

**Задание 1. Посчитаем, сколько книг вышло после 1 января 2000 года**

In [22]:
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01';
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


,count
0,819


**Вывод:** После 1 января 2000 года вышло `819` книг.

**Задание 2. Для каждой книги посчитаем количество обзоров и среднюю оценку**

In [23]:
query = '''
SELECT title,
       COUNT(DISTINCT review_id) AS count_review,
       ROUND(AVG(rating), 2) AS rating_avg
FROM books AS b 
     LEFT JOIN ratings AS r 
     ON b.book_id = r.book_id
     LEFT JOIN reviews AS rv 
     ON b.book_id = rv.book_id
GROUP BY b.book_id
ORDER BY rating_avg DESC, count_review DESC;
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,title,count_review,rating_avg
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


,title,count_review,rating_avg
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


**Вывод:** Отсортировали 1000 книг по количеству обзоров и средней оценке. Книга с наибольшим рейтингом и количеством обзоров - `A Dirty Job (Grim Reaper #1)`.

**Задание 3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры**

In [24]:
query = '''
SELECT publishers.publisher,
       COUNT(books.book_id) AS count_books
FROM publishers
LEFT JOIN books ON publishers.publisher_id = books.publisher_id
WHERE num_pages > 50
GROUP BY publishers.publisher
ORDER BY count_books DESC
LIMIT 1;
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count_books
0,Penguin Books,42


,publisher,count_books
0,Penguin Books,42


**Вывод:** Издательство `Penguin Books` выпустило 42 книги толще 50 страниц.

**Задание 4. Определим автора с самой высокой средней оценкой книг — учтём только книги с 50 и более оценками**

In [25]:
query = '''
SELECT 
    author,
    AVG(rating) AS avg_rating
FROM books
LEFT JOIN ratings on books.book_id = ratings.book_id
LEFT JOIN authors on books.author_id = authors.author_id
WHERE
    books.book_id IN (
    SELECT ratings.book_id
    FROM ratings
    GROUP BY ratings.book_id
    HAVING COUNT(rating) >= 50)
GROUP BY author
ORDER BY avg_rating DESC
LIMIT 1;
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


**Вывод:** `J.K. Rowling/Mary GrandPré` - aвтор книг с самой высокой средней оценкой `4.287097` среди книг, которым поставили 50 и более оценок.

**Задание 5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок**

In [26]:
query = '''
SELECT AVG(r.count) AS avg_review
FROM 
    (SELECT count(*)
     FROM reviews
     WHERE username IN
         (SELECT username
          FROM ratings
          GROUP BY username
          HAVING count(rating) > 48)
    GROUP BY username) AS r;
'''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_review
0,24.0


,avg_review
0,24.0


**Вывод:** Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - `24`.

**Вывод по разделу 2:**

- После 1 января 2000 года вышло `819` книг.
- Отсортировали 1000 книг по количеству обзоров и средней оценке. Книга с наибольшим рейтингом и количеством обзоров - `A Dirty Job (Grim Reaper #1)`.
- Издательство `Penguin Books` выпустило 42 книги толще 50 страниц.
- `J.K. Rowling/Mary GrandPré` - aвтор книг с самой высокой средней оценкой `4.287097` среди книг, которым поставили 50 и более оценок.
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - `24`.

#### Шаг 3. Общий вывод по всей работе

**Вывод по разделу 1:**

**Таблица `books`**

Содержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.


**Вывод по разделу 2:**

- После 1 января 2000 года вышло `819` книг.
- Отсортировали 1000 книг по количеству обзоров и средней оценке. Книга с наибольшим рейтингом и количеством обзоров - `A Dirty Job (Grim Reaper #1)`.
- Издательство `Penguin Books` выпустило 42 книги толще 50 страниц.
- `J.K. Rowling/Mary GrandPré` - aвтор книг с самой высокой средней оценкой `4.287097` среди книг, которым поставили 50 и более оценок.
- Среднее количество обзоров от пользователей, которые поставили больше 48 оценок - `24`.